In [283]:
%LOAD pizza_runner_changed.db rw

"Before you start writing your SQL queries however - you might want to investigate the data, you may want to do something with some of those null values and data types in the customer_orders and runner_orders tables!"

## Tables

In [284]:
SELECT name, sql
FROM sqlite_schema
WHERE type ='table';

name,sql
runners,"CREATE TABLE runners ( ""runner_id"" INTEGER, ""registration_date"" DATE )"
customer_orders,"CREATE TABLE customer_orders ( ""order_id"" INTEGER, ""customer_id"" INTEGER, ""pizza_id"" INTEGER, ""exclusions"" VARCHAR(4), ""extras"" VARCHAR(4), ""order_time"" TIMESTAMP )"
runner_orders,"CREATE TABLE runner_orders ( ""order_id"" INTEGER, ""runner_id"" INTEGER, ""pickup_time"" VARCHAR(19), ""distance"" VARCHAR(7), ""duration"" VARCHAR(10), ""cancellation"" VARCHAR(23) )"
pizza_names,"CREATE TABLE pizza_names ( ""pizza_id"" INTEGER, ""pizza_name"" TEXT )"
pizza_recipes,"CREATE TABLE pizza_recipes ( ""pizza_id"" INTEGER, ""toppings"" TEXT )"
pizza_toppings,"CREATE TABLE pizza_toppings ( ""topping_id"" INTEGER, ""topping_name"" TEXT )"


In [285]:
PRAGMA table_info(runner_orders);

cid,name,type,notnull,dflt_value,pk
0,order_id,INTEGER,0,,0
1,runner_id,INTEGER,0,,0
2,pickup_time,VARCHAR(19),0,,0
3,distance,VARCHAR(7),0,,0
4,duration,VARCHAR(10),0,,0
5,cancellation,VARCHAR(23),0,,0


### Table customer_orders

In [286]:
DROP VIEW clean_customer_orders;

In [287]:
CREATE VIEW clean_customer_orders AS

WITH with_rows AS (
    SELECT ROW_NUMBER () OVER ( 
    ORDER BY order_id 
    ) unique_id, *
    FROM customer_orders),

clean_exclusions AS (
    SELECT unique_id,
        CASE WHEN exclusions IN ('null', '') THEN NULL 
        ELSE exclusions END AS exclusions
    FROM with_rows),

clean_extras AS (
    SELECT unique_id,
        CASE WHEN extras IN ('null', '') THEN NULL 
        ELSE extras END AS extras
    FROM with_rows)
    
SELECT wr.unique_id, wr.order_id, wr.customer_id, wr.pizza_id, wr.order_time,
        ce_minus.exclusions, ce_plus.extras
FROM with_rows AS wr
INNER JOIN clean_exclusions AS ce_minus
USING(unique_id)
INNER JOIN clean_extras AS ce_plus
USING(unique_id)

### Table runner_orders

In [288]:
DROP VIEW clean_runner_orders;

In [289]:
CREATE VIEW clean_runner_orders AS

WITH clean_pickup_time AS (
    SELECT order_id, datetime(
        CASE WHEN pickup_time IN ('null', '') THEN NULL 
        ELSE pickup_time END) AS pickup_time
    FROM runner_orders),
    
clean_distance AS (
    SELECT order_id, CAST(
        CASE WHEN distance IN ('null', '') THEN NULL
        ELSE distance END AS INT) AS distance
    FROM runner_orders),
    
clean_duration AS (
    SELECT order_id, CAST(
        CASE WHEN duration IN ('null', '') THEN NULL
        ELSE duration END AS INT) AS duration
    FROM runner_orders),
    
clean_cancellation AS (
    SELECT order_id, CAST(
        CASE WHEN cancellation IN ('null', '') THEN NULL
        ELSE cancellation END AS VARCHAR(20)) AS cancellation
    FROM runner_orders)
    
SELECT ro.order_id, ro.runner_id, 
        cpt.pickup_time, CAST(cdi.distance AS INT), cdu.duration, cc.cancellation
FROM runner_orders AS ro
INNER JOIN clean_pickup_time AS cpt
USING (order_id)
INNER JOIN clean_duration AS cdu
USING (order_id)
INNER JOIN clean_distance AS cdi
USING (order_id)
INNER JOIN clean_cancellation AS cc
USING (order_id);

## A. Pizza Metrics

### 1. How many pizzas were ordered?

In [290]:
SELECT COUNT(pizza_id) AS nr_of_pizzas
FROM clean_customer_orders;

nr_of_pizzas
14


### 2.How many unique customer orders were made?
I assume that we are looking for unique combinations of pizza_id, exclusions, and extras.

In [293]:
WITH distinct_orders AS (
    SELECT DISTINCT pizza_id, exclusions, extras
    FROM clean_customer_orders
    )
    
SELECT COUNT(*) AS unique_customer_orders
FROM distinct_orders;

unique_customer_orders
8


### 3. How many successful orders were delivered by each runner?

In [299]:
SELECT runner_id, COUNT(*) AS nr_successful_orders
FROM clean_runner_orders
WHERE cancellation IS NULL
GROUP BY runner_id

runner_id,nr_successful_orders
1,4
2,3
3,1


### 4. How many of each type of pizza was delivered?

In [309]:
WITH successful_deliveries AS (
    SELECT customer.*
    FROM clean_runner_orders AS runner
    INNER JOIN clean_customer_orders AS customer
    USING(order_id)
    WHERE runner.cancellation IS NULL)

SELECT pn.pizza_name, COUNT(sd.order_id) AS nr_pizzas
FROM successful_deliveries AS sd
INNER JOIN pizza_names AS pn
USING(pizza_id)
GROUP BY sd.pizza_id

pizza_name,nr_pizzas
Meatlovers,9
Vegetarian,3


### 5. How many Vegetarian and Meatlovers were ordered by each customer?

In [322]:
SELECT co.customer_id, pn.pizza_name, COUNT(*) AS nr_pizzas
FROM customer_orders AS co
INNER JOIN pizza_names AS pn
USING(pizza_id)
GROUP BY customer_id, pizza_id

customer_id,pizza_name,nr_pizzas
101,Meatlovers,2
101,Vegetarian,1
102,Meatlovers,2
102,Vegetarian,1
103,Meatlovers,3
103,Vegetarian,1
104,Meatlovers,3
105,Vegetarian,1


### 6. What was the maximum number of pizzas delivered in a single order?
I will only consider successful deliveries.

In [329]:
WITH successful_deliveries AS (
    SELECT customer.*
    FROM clean_runner_orders AS runner
    INNER JOIN clean_customer_orders AS customer
    USING(order_id)
    WHERE runner.cancellation IS NULL)
    
SELECT COUNT(order_id) AS max_of_pizzas_per_delivery
FROM successful_deliveries AS sd
GROUP BY order_id
ORDER BY max_of_pizzas_per_delivery DESC
LIMIT 1;

max_of_pizzas_per_delivery
3


### 7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?
I will assume that changes are either exclusions or extras. I will only consider successful deliveries.

In [357]:
WITH successful_deliveries AS (
    SELECT customer.*
    FROM clean_runner_orders AS runner
    INNER JOIN clean_customer_orders AS customer
    USING(order_id)
    WHERE runner.cancellation IS NULL),
    
has_exclusions AS (
    SELECT unique_id,
        CASE WHEN exclusions IS NULL THEN 0
        ELSE 1 END AS minus
    FROM successful_deliveries),

has_extras AS (
    SELECT unique_id,
        CASE WHEN extras IS NULL THEN 0
        ELSE 1 END AS plus
    FROM successful_deliveries),
    
col_changes AS (
    SELECT sd.*, 
        CASE WHEN MAX(hp.plus, hm.minus) = 1 THEN 'At least 1 change'
        ELSE 'No changes' END AS val
    FROM successful_deliveries AS sd
    INNER JOIN has_exclusions AS hm
    USING (unique_id)
    INNER JOIN has_extras AS hp
    USING (unique_id) )
    
SELECT customer_id, val, COUNT(unique_id) AS nr_pizzas
FROM col_changes
GROUP BY customer_id, val;

customer_id,val,nr_pizzas
101,No changes,2
102,No changes,3
103,At least 1 change,3
104,At least 1 change,2
104,No changes,1
105,At least 1 change,1


### 8. How many pizzas were delivered that had both exclusions and extras?

In [366]:
WITH successful_deliveries AS (
    SELECT customer.*
    FROM clean_runner_orders AS runner
    INNER JOIN clean_customer_orders AS customer
    USING(order_id)
    WHERE runner.cancellation IS NULL),
    
has_exclusions AS (
    SELECT unique_id,
        CASE WHEN exclusions IS NULL THEN 0
        ELSE 1 END AS minus
    FROM successful_deliveries),

has_extras AS (
    SELECT unique_id,
        CASE WHEN extras IS NULL THEN 0
        ELSE 1 END AS plus
    FROM successful_deliveries)
    
SELECT COUNT(unique_id) AS nr_pizzas_with_exlusions_extras
FROM successful_deliveries AS sd
INNER JOIN has_exclusions AS hm
USING (unique_id)
INNER JOIN has_extras AS hp
USING (unique_id) 
WHERE hp.plus + hm.minus > 1

nr_pizzas_with_exlusions_extras
1


### 9. What was the total volume of pizzas ordered for each hour of the day?

In [375]:
WITH with_hour AS (
    SELECT *, strftime('%H', order_time) AS hour
    FROM clean_customer_orders)
    
SELECT hour, COUNT(unique_id) AS nr_pizzas
FROM with_hour
GROUP BY hour

hour,nr_pizzas
11,1
13,3
18,3
19,1
21,3
23,3


### 10. What was the volume of orders for each day of the week?

In [390]:
WITH with_day AS (
    SELECT *,
    CASE WHEN strftime('%w', order_time) = '0' THEN 'Sunday'
    WHEN strftime('%w', order_time) = '1' THEN 'Monday'
    WHEN strftime('%w', order_time) = '2' THEN 'Tuesday'
    WHEN strftime('%w', order_time) = '3' THEN 'Wednesday'
    WHEN strftime('%w', order_time) = '4' THEN 'Thursday'
    WHEN strftime('%w', order_time) = '5' THEN 'Friday'
    WHEN strftime('%w', order_time) = '6' THEN 'Saturday' 
    ELSE '' END AS day
    FROM clean_customer_orders)
    
SELECT day, COUNT(unique_id) AS nr_pizzas
FROM with_day
GROUP BY day


day,nr_pizzas
Friday,1
Saturday,5
Thursday,3
Wednesday,5


## B. Runner and Customer Experience

### 1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [393]:
select * from runners

runner_id,registration_date
1,2021-01-01
2,2021-01-03
3,2021-01-08
4,2021-01-15


In [403]:
select LAG(date('2021-01-01', '+7 day')) OVER (ORDER BY registration_date)
FROM runners



"LAG(date('2021-01-01', '+7 day')) OVER (ORDER BY registration_date)"
""
2021-01-08
2021-01-08
2021-01-08
